In [37]:
# node_base.py
from typing import Any, Dict, Type, Union
from pydantic import BaseModel

from abc import ABC, abstractmethod
import json


class ParameterSpec(BaseModel):
    """Describe one config‑parameter: its type, default value, and an optional description."""

    type: Type
    default: Any
    description: str = ""


class NodeSpec(BaseModel):
    name: str
    description: str
    inputs: Dict[str, type]
    outputs: Dict[str, type]
    parameters: Dict[str, ParameterSpec]


class Node(ABC):
    spec: NodeSpec

    def __init__(self, **kwargs):
        # validate parameters, set attributes…
        pass

    @abstractmethod
    def run(self, **inputs):
        """Override in subclasses: consume inputs & parameters → produce outputs."""
        raise NotImplementedError

    def get_spec_json(self) -> Dict[str, Any]:
        """
        Return a JSON‑serializable dict of this node's spec,
        converting Python types and Pydantic models into strings/dicts.
        """

        def _serialize_type(t: Type) -> Union[str, Dict[str, Any]]:
            # 1) built‐ins and custom classes → just type name
            if not isinstance(t, type):
                return str(t)
            name = t.__name__
            # 2) Pydantic models → include their own JSON schema
            if issubclass(t, BaseModel):
                return {
                    "type": name,
                    "schema": t.model_json_schema(),  # Pydantic’s JSON Schema for your model
                }
            # 3) otherwise (int, str, float, etc.) → just the name
            return name

        raw = self.spec.model_dump()
        raw["inputs"] = {
            port: _serialize_type(tp) for port, tp in self.spec.inputs.items()
        }
        raw["outputs"] = {
            port: _serialize_type(tp) for port, tp in self.spec.outputs.items()
        }
        # parameters are assumed JSON‐safe already
        return raw

    def get_spec_json_str(self) -> str:
        return json.dumps(self.get_spec_json(), indent=2)


class CustomType(BaseModel):
    name: str
    description: str


class ChatInput(Node):
    spec: NodeSpec = NodeSpec(
        name="Chat Input",
        description="A node that accepts user input and returns a message.",
        inputs={},
        outputs={"message": str, "test": int},
        parameters={},
    )

    def run(self, **inputs):
        return inputs["message"]


node = ChatInput()
node.get_spec_json()

{'name': 'Chat Input',
 'description': 'A node that accepts user input and returns a message.',
 'inputs': {},
 'outputs': {'message': 'str', 'test': 'int'},
 'parameters': {}}

In [43]:
# node_base.py
from abc import ABC, abstractmethod
from typing import Any, Dict, Type

from pydantic import BaseModel


class ParameterSpec(BaseModel):
    """Describe a single node‑parameter: its type, default value, and description."""

    type: Type
    default: Any
    description: str = ""


class NodeSpec(BaseModel):
    name: str
    description: str
    inputs: Dict[str, Type]
    outputs: Dict[str, Type]
    parameters: Dict[str, ParameterSpec]


class Node(ABC):
    spec: NodeSpec

    @abstractmethod
    def run(self, **inputs) -> Any: ...

    def get_spec_json(self) -> Dict[str, Any]:
        """
        Return a JSON‑serializable dict describing:
         - name, description
         - inputs  (port → type name)
         - outputs (port → type name)
         - parameters (param → {type name, default, description})
        """

        def _serialize_type(t: Type) -> Any:
            # If it's a Pydantic model, embed its JSON schema:
            if isinstance(t, type) and issubclass(t, BaseModel):
                return {"type": t.__name__, "schema": t.model_json_schema()}
            # Otherwise just return the class name:
            return getattr(t, "__name__", str(t))

        raw = self.spec.dict()  # gives you a dict with raw types still in it
        # replace inputs/outputs with their stringified versions:
        raw["inputs"] = {k: _serialize_type(v) for k, v in self.spec.inputs.items()}
        raw["outputs"] = {k: _serialize_type(v) for k, v in self.spec.outputs.items()}

        # rebuild parameters into a JSON‑safe form:
        params = {}
        for name, p in self.spec.parameters.items():
            params[name] = {
                "type": _serialize_type(p.type),
                "default": p.default,
                "description": p.description,
            }
        raw["parameters"] = params

        return raw

    def get_spec_json_str(self) -> str:
        """Pretty‑print the above dict as JSON."""
        return json.dumps(self.get_spec_json(), indent=2)


class CustomType(BaseModel):
    name: str
    description: str


class ThresholdFilter(Node):
    spec = NodeSpec(
        name="Threshold Filter",
        description="Filters numeric input based on a threshold.",
        inputs={"value": float},
        outputs={"passed": bool},
        parameters={
            "threshold": ParameterSpec(
                type=float, default=10.0, description="Values ≥ threshold will pass"
            )
        },
    )

    def run(self, **inputs):
        val = inputs["value"]
        thresh = self.spec.parameters["threshold"].default
        return {"passed": val >= thresh}


if __name__ == "__main__":
    node = ThresholdFilter()
    print(node.get_spec_json_str())

{
  "name": "Threshold Filter",
  "description": "Filters numeric input based on a threshold.",
  "inputs": {
    "value": "float"
  },
  "outputs": {
    "passed": "bool"
  },
  "parameters": {
    "threshold": {
      "type": "float",
      "default": 10.0,
      "description": "Values \u2265 threshold will pass"
    }
  }
}


/tmp/ipykernel_1581585/2976896014.py:49: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  raw = self.spec.dict()  # gives you a dict with raw types still in it
